Part 1: Loading the libraries, the dataset, and getting aquainted with the data structure.

In [91]:
import pandas as pd
import numpy as np
import json

In [92]:
#load dataset
df = pd.read_csv("dataset/movies_dataset.csv", encoding="utf-8")

C:\Users\ByakkoMao\AppData\Local\Temp\ipykernel_8392\4062744367.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset/movies_dataset.csv", encoding="utf-8")


In [93]:
#checking how much data we are dealing with
df.shape

(45466, 24)

In [94]:
#quick view of what the dataframe looks like.
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [95]:
#checking classes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Part 2: Now that we know more or less what the df looks like, it's time to start working with it.
Intuitively, the first column we are going to try to fix is 'belongs_to_collection'. This is the first column in the dataframe that needs obvious attention.

In [96]:
#function to use json.loads and avoid errors

def load_json(x):
    try:
        return json.loads(x)
    except (json.JSONDecodeError, TypeError):
        return {}


In [97]:
#adjusting the data inside the columns, changing the '' encasing to "" for the load_json function

df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: str(x).replace("'", '"') if isinstance(x, str) else '')



In [98]:
#load_json to remove columns and add a new column with the required data, but produces even more unclean data.
#for some reason, if the load_json was applied directly, it kept giving out errors, but with the created function it works and nothing seemed to be too broken.

df['belongs_to_collection'] = df['belongs_to_collection'].apply(load_json)
df_belongs = pd.json_normalize(df['belongs_to_collection'])
df_belongs = df_belongs.reset_index(drop=True)
df = pd.concat([df, df_belongs], axis=1)
df.drop(columns=['belongs_to_collection'], inplace=True)

In [99]:
#we can check the current progress, to see what the extracted data looks like, and what further changes are needed

df.head()

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,status,tagline,title,video,vote_average,vote_count,id,name,poster_path,backdrop_path
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,NaN
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


It was necessary to do it this way, because with the function 'extract_nested_data', that is written below, the entire column turned into empty values.
With the previous code, at least we have some of the collections. Why they won't let us drop this column altogether instead is beyond me, but here it is.
Now that the column turned into 3 separate ones at the end of the df, we can these as needed.
Unfortunately, we now have a second column called id, which is in our best interest to remove it so as to not cause any conflict with the original id column.
Because they both have the same name, it's important to drop the correct column. this is done by finding the index value of the column we want to erase.

In [100]:
#first, we make sure we have the correct index for the extra id column, as we know it was near the end of the dataframe.
col_name = df.columns[23]
print(col_name)

id


In [101]:
#now we can "safely" erase the id column that isn't needed.
df.drop(df.columns[23], axis=1, inplace=True)


Part 3: Continuing with the rest of the nested data.
This time we can use a function to extract the value of the name within the rest of the nested dictionaries.
We are returning the data as a list, because if you try to modify the column values in the future, it will be easier to add data to a list, rather than a string.
This also avoids having to convert the strings into a list before modifying it, or potentially losing the entire string. 

THIS ONE RETURNS A LIST, THE FUNCTION BELOW RETURNS A STRING, RUN ONE OR THE OTHER, NOT BOTH

def extract_nested_data(names):
    try:
        object_list = eval(names)
        object_names = [object_dict['name'] for object_dict in object_list]
        return object_names
    except:
        return ''

In [102]:
#THIS ONE RETURNS A STRING, NOT A LIST, IF YOU WANT THE DATA WITHIN THE COLUMNS SET AS A LIST, RUN THE ONE ABOVE

def extract_nested_data(names):
    try:
        object_list = eval(names)
        object_names = [object_dict['name'] for object_dict in object_list]
        return ', '.join(object_names)
    except:
        return ''

In [103]:
#applying the function to the columns that had nested data
df['production_countries'] = df['production_countries'].apply(extract_nested_data)
df['production_companies'] = df['production_companies'].apply(extract_nested_data)
df['genres'] = df['genres'].apply(extract_nested_data)
df['spoken_languages'] = df['spoken_languages'].apply(extract_nested_data)

In [104]:
#turning release_date into date time so we can put the year in a separate column.
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

In [105]:
#checking the release years sorted as array, that lets us see from when are the latest movies
np.sort(df.release_year.unique())

array([1874., 1878., 1883., 1887., 1888., 1890., 1891., 1892., 1893.,
       1894., 1895., 1896., 1897., 1898., 1899., 1900., 1901., 1902.,
       1903., 1904., 1905., 1906., 1907., 1908., 1909., 1910., 1911.,
       1912., 1913., 1914., 1915., 1916., 1917., 1918., 1919., 1920.,
       1921., 1922., 1923., 1924., 1925., 1926., 1927., 1928., 1929.,
       1930., 1931., 1932., 1933., 1934., 1935., 1936., 1937., 1938.,
       1939., 1940., 1941., 1942., 1943., 1944., 1945., 1946., 1947.,
       1948., 1949., 1950., 1951., 1952., 1953., 1954., 1955., 1956.,
       1957., 1958., 1959., 1960., 1961., 1962., 1963., 1964., 1965.,
       1966., 1967., 1968., 1969., 1970., 1971., 1972., 1973., 1974.,
       1975., 1976., 1977., 1978., 1979., 1980., 1981., 1982., 1983.,
       1984., 1985., 1986., 1987., 1988., 1989., 1990., 1991., 1992.,
       1993., 1994., 1995., 1996., 1997., 1998., 1999., 2000., 2001.,
       2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009., 2010.,
       2011., 2012.,

Part 4: we can now start removing the data that will not be in use.
This includes dropping unnecessary columns, checking for missing values and completing accordingly, and dropping potential duplicates.
Since we had to drop vote count as part of the requests from this project, popularity, as well as the vote average feel a bit worthless. while they could have been used as metric for something when combined, by themselves are of no use. so they are going to the drop pile.

In [106]:
#dropping all the columns that won't be in use, as per asked for this project, adding status to the list, because it's redundant. 
df.drop(columns=['video', 'imdb_id', 'status',
                'adult', 'original_title', 'vote_count', 'poster_path', 'backdrop_path',
                'popularity', 'vote_average' ,'homepage'], inplace=True)

In [107]:
#checking the null values from the remaining columns
df.isnull().sum()

budget                      0
genres                      0
original_language          11
overview                  954
production_companies        0
production_countries        0
release_date               90
revenue                     6
runtime                   263
spoken_languages            0
tagline                 25054
title                       6
name                    42298
release_year               90
dtype: int64

In [108]:
#not even going to check if there is any, just straight up dropping duplicates is enough in this case. this is might not be the case elsewhere though.
#df.drop_duplicates()

In [109]:
#dropping these two that have missing values because they might cause problems in the future.
df.dropna(subset=['title'], inplace=True)
df.dropna(subset=['release_date'], inplace=True)

In [110]:
#budget wasn't a numeric column, and just in case, we are also fixing revenue as well, to convert and add the missing values on both columns.
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce').astype(float).fillna(0)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce').astype(float).fillna(0)

In [111]:
#basic profit calculation. we are missing a total expenditures column, so we can't properly make a RoI analysis, so instead we just get the basic return of each movie.

#we make the column 'return' be = 0 always.
df['return'] = 0
for i in range(len(df)):
#unless both revenue and budget column are distinct from 0, the return column remains as 0
    if i in df.index and df['revenue'][i] != 0 and df['budget'][i] != 0:
#when both have have numbers distinct from 0, it continues on and divides revenue by budget, updating the return column.
        df.loc[i, 'return'] = df['revenue'][i] / df['budget'][i]



In [112]:
#adjusting the name of data extracted from the original belong_to_collection column.
df = df.rename(columns={'name': 'belongs_to_collection'})
df['belongs_to_collection'] = df['belongs_to_collection'].fillna('')

#completing the rest of the missing data
df['tagline'] = df['tagline'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['status'] = df['status'].fillna("No Status")
df['original_language'] = df['original_language'].fillna("Unknown")
df['spoken_languages'] = df['spoken_languages'].fillna("Unkown")
df['overview'] = df['overview'].fillna('')

In [113]:
#cleaner completion of missing data. keeping them empty wont skew the algorithm for the recommendations.
df.fillna({ 'tagline':'',
            'runtime': 0, 
            'status': '',
            'original_language': '', 
            'spoken_languages': '',
            'overview':''
            }, inplace=True)

In [114]:
#quick check to see the work so far
df.head()

,budget,genres,original_language,overview,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,belongs_to_collection,release_year,return
0,30000000.0,"Animation, Comedy, Family",en,"Led by Woody, Andy's toys live happily in his ...",Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,,Toy Story,Toy Story Collection,1995.0,12.451801
1,65000000.0,"Adventure, Fantasy, Family",en,When siblings Judy and Peter discover an encha...,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,1995-12-15,262797249.0,104.0,"English, Français",Roll the dice and unleash the excitement!,Jumanji,,1995.0,4.043035
2,0.0,"Romance, Comedy",en,A family wedding reignites the ancient feud be...,"Warner Bros., Lancaster Gate",United States of America,1995-12-22,0.0,101.0,English,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,Grumpy Old Men Collection,1995.0,0.000000
3,16000000.0,"Comedy, Drama, Romance",en,"Cheated on, mistreated and stepped on, the wom...",Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,English,Friends are the people who let you be yourself...,Waiting to Exhale,,1995.0,5.090760
4,0.0,Comedy,en,Just when George Banks has recovered from his ...,"Sandollar Productions, Touchstone Pictures",United States of America,1995-02-10,76578911.0,106.0,English,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,Father of the Bride Collection,1995.0,0.000000


In [115]:
#checking the data types of the columns as reference
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   budget                 45376 non-null  float64       
 1   genres                 45376 non-null  object        
 2   original_language      45376 non-null  object        
 3   overview               45376 non-null  object        
 4   production_companies   45376 non-null  object        
 5   production_countries   45376 non-null  object        
 6   release_date           45376 non-null  datetime64[ns]
 7   revenue                45376 non-null  float64       
 8   runtime                45376 non-null  float64       
 9   spoken_languages       45376 non-null  object        
 10  tagline                45376 non-null  object        
 11  title                  45376 non-null  object        
 12  belongs_to_collection  45376 non-null  object        
 13  r

Part 5: We now have a workable dataframe. However it makes my eyes bleed.
Everything from this point on is just pure aesthetic work to make the final dataset look better, there is nothing functional about this anymore.
To start with, we are turning every column into an object class, to avoid issues when reindexing.
After reindexing, we give every column it's proper class once again.

In [116]:
#convert all columns to object class
df = df.astype('object')

In [117]:
#reindex the columns
df = df.reindex(columns=['title','overview','belongs_to_collection', 
                        'production_companies','production_countries',
                        'original_language','spoken_languages','genres',
                        'tagline','release_date','release_year',
                        'runtime','revenue','budget','return'])


In [118]:
#convert the columns back to their original classes
df['runtime'] = df['runtime'].astype('float')
df['revenue'] = df['revenue'].astype('float')
df['budget'] = df['budget'].astype('float')
df['return'] = df['return'].astype('float')
#we are reusing these two following lines because it was just easier to do this twice than do it now and then rearanging the release year to be next of release_date.
#not optimal, but not a big deal because it doesnt take long anyways.
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

In [119]:
#we check the work one last time to make sure everything got set properly.
df.head()

,title,overview,belongs_to_collection,production_companies,production_countries,original_language,spoken_languages,genres,tagline,release_date,release_year,runtime,revenue,budget,return
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Toy Story Collection,Pixar Animation Studios,United States of America,en,English,"Animation, Comedy, Family",,1995-10-30,1995,81.0,373554033.0,30000000.0,12.451801
1,Jumanji,When siblings Judy and Peter discover an encha...,,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,en,"English, Français","Adventure, Fantasy, Family",Roll the dice and unleash the excitement!,1995-12-15,1995,104.0,262797249.0,65000000.0,4.043035
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,en,English,"Romance, Comedy",Still Yelling. Still Fighting. Still Ready for...,1995-12-22,1995,101.0,0.0,0.0,0.000000
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",,Twentieth Century Fox Film Corporation,United States of America,en,English,"Comedy, Drama, Romance",Friends are the people who let you be yourself...,1995-12-22,1995,127.0,81452156.0,16000000.0,5.090760
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,en,English,Comedy,Just When His World Is Back To Normal... He's ...,1995-02-10,1995,106.0,76578911.0,0.0,0.000000


In [120]:
df.isnull().count()

title                    45376
overview                 45376
belongs_to_collection    45376
production_companies     45376
production_countries     45376
original_language        45376
spoken_languages         45376
genres                   45376
tagline                  45376
release_date             45376
release_year             45376
runtime                  45376
revenue                  45376
budget                   45376
return                   45376
dtype: int64

In [121]:
df.isna().count()

title                    45376
overview                 45376
belongs_to_collection    45376
production_companies     45376
production_countries     45376
original_language        45376
spoken_languages         45376
genres                   45376
tagline                  45376
release_date             45376
release_year             45376
runtime                  45376
revenue                  45376
budget                   45376
return                   45376
dtype: int64

OPTIONAL: we could turn the larger numbers into more readable strings, albeit at the cost of them losing the float class, making it impossible to do future operations with them.
it looks a lot better, however we have to work on the numbers again with the API, so it's something that might get added there instead.
Here is the code that will be used in that case:

#transform larger numbers to make them more readable, but keeps them as string.
locale.setlocale(locale.LC_ALL, 'en_US')
df['revenue'] = df['revenue'].apply(lambda x: locale.format_string("%d", x, grouping=True))
df['budget'] = df['budget'].apply(lambda x: locale.format_string("%d", x, grouping=True))
df['return'] = df['return'].apply(lambda x: locale.format_string("%d", x, grouping=True))

# convert columns back to float again
df['revenue'] = df['revenue'].str.replace(',', '').astype(float)
df['budget'] = df['budget'].str.replace(',', '').astype(float)
df['return'] = df['return'].str.replace(',', '').astype(float)


In [123]:
#last step is to conver the final dataframe into a separate csv dataset.
df.to_csv('dataset/clean_dataset.csv', index=False)